# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686567


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:20,  2.76s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:07,  2.42s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:46,  1.73s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:33,  1.30s/it]

...

Rendering models:  81%|████████  | 25/31 [00:09<00:00,  7.21it/s]

Rendering models:  84%|████████▍ | 26/31 [00:09<00:00,  7.56it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  8.10it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  9.27it/s]

Anfrily                               0.000154
not-logged-in-73a441ca06acb0ea9b0a    0.000112
Dina72                                0.000660
CThomas                               0.000859
suncoup                               0.000090
not-logged-in-5f1b09619258b45f64fe    0.001096
not-logged-in-2159616075e1803d6325    0.000420
thedarktree                           0.000130
jmfranci                              0.000227
MilesGray                             0.000516
pangeli5                              0.000596
acapirala                             0.000088
Lavadude                              0.011554
Mr_DMan                               0.000466
not-logged-in-5ab66acd5f2b804399bb    0.000289
not-logged-in-340a28051a320c10499e    0.000150
FR4N1O                                0.000777
tingard                               0.000088
ckkmomma                              0.003206
not-logged-in-7b7bc2eb0877c9ffcdd6    0.000230
not-logged-in-8fc584188a90e1b9c7a1    0.001155
spannerfarm  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())